In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tabular-playground-series-apr-2021/sample_submission.csv
/kaggle/input/tabular-playground-series-apr-2021/train.csv
/kaggle/input/tabular-playground-series-apr-2021/test.csv


**************************************************

## TO DO

### ~~Scaling numerical columns~~
### Using Kfold - Average - predict_proba
### Increase Trials
### ~~Range of values for n_estimators~~
### Threshold for predict_proba

**************************************

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
import optuna
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler

In [3]:
train = pd.read_csv("/kaggle/input/tabular-playground-series-apr-2021/train.csv")
test = pd.read_csv("/kaggle/input/tabular-playground-series-apr-2021/test.csv")
submission = pd.read_csv("/kaggle/input/tabular-playground-series-apr-2021/sample_submission.csv")

In [4]:
display(train.head())
display(test.head())

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0,1,1,"Oconnor, Frankie",male,NaN,2,0,209245,27.14,C12239,S
1,1,0,3,"Bryan, Drew",male,NaN,0,0,27323,13.35,NaN,S
2,2,0,3,"Owens, Kenneth",male,0.33,1,2,CA 457703,71.29,NaN,S
3,3,0,3,"Kramer, James",male,19.00,0,0,A. 10866,13.04,NaN,S
4,4,1,3,"Bond, Michael",male,25.00,0,0,427635,7.76,NaN,S


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,100000,3,"Holliday, Daniel",male,19.0,0,0,24745,63.01,NaN,S
1,100001,3,"Nguyen, Lorraine",female,53.0,0,0,13264,5.81,NaN,S
2,100002,1,"Harris, Heather",female,19.0,0,0,25990,38.91,B15315,C
3,100003,2,"Larsen, Eric",male,25.0,0,0,314011,12.93,NaN,S
4,100004,1,"Cleary, Sarah",female,17.0,0,2,26203,26.89,B22515,C


In [5]:
df = train.append(test, sort=False)
train_len = len(train)

In [6]:
df.isnull().sum()

PassengerId         0
Survived       100000
Pclass              0
Name                0
Sex                 0
Age              6779
SibSp               0
Parch               0
Ticket           9804
Fare              267
Cabin          138697
Embarked          527
dtype: int64

In [7]:
df['Fare'] = df['Fare'].fillna(df['Fare'].mean())

df['Family_Size'] = df['SibSp'] + df['Parch'] + 1

df['Name'] = df['Name'].map(lambda x: x.split(',')[0])

df['isAlone'] = df['Family_Size'].map(lambda x: 1 if x==1 else 0)

# FE : https://www.kaggle.com/jmargni/tps-apr-2021-lightgbm-cv

age_map = df[['Age', 'Pclass']].dropna().groupby('Pclass').mean().to_dict()
df['Age'] = df['Age'].fillna(df['Pclass'].map(age_map['Age']))

df['Cabin'] = df['Cabin'].fillna('X').map(lambda x: x[0].strip())

df['Ticket'] = df['Ticket'].fillna('X').map(lambda x: str(x).split()[0] if len(str(x).split()) > 1 else 'X')

df['Embarked'] = df['Embarked'].fillna('X')

In [8]:
df.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Family_Size,isAlone
0,0,1.0,1,Oconnor,male,40.672757,2,0,X,27.14,C,S,3,0
1,1,0.0,3,Bryan,male,30.205570,0,0,X,13.35,X,S,1,1
2,2,0.0,3,Owens,male,0.330000,1,2,CA,71.29,X,S,4,0
3,3,0.0,3,Kramer,male,19.000000,0,0,A.,13.04,X,S,1,1
4,4,1.0,3,Bond,male,25.000000,0,0,X,7.76,X,S,1,1
5,5,0.0,2,Sassano,male,35.000000,0,0,X,6.71,X,S,1,1
6,6,0.0,3,Conway,male,7.000000,0,0,X,9.77,X,S,1,1
7,7,1.0,3,Werner,female,6.000000,1,2,X,31.50,X,S,4,0
8,8,0.0,1,Wardlaw,male,27.000000,2,1,X,73.02,A,S,4,0
9,9,0.0,2,Greigo,male,66.000000,0,0,X,9.14,D,C,1,1


In [9]:
df.drop(['PassengerId', 'Survived'], axis=1, inplace=True)

In [10]:
label_col = [['Name'], ["Ticket"]]
cat_col = ['Sex', 'Cabin', 'Embarked']
num_col = [x for x in df.columns if x not in (cat_col + label_col)]
# numerical_col, categorical_col
num_col

['Pclass',
 'Name',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Family_Size',
 'isAlone']

In [11]:
lc = LabelEncoder()
scaler = StandardScaler()
df_copy = df.copy()
# df[label_col]

In [12]:
df_copy = pd.get_dummies(df, columns=cat_col)
df_copy[['Name']] = lc.fit_transform(df_copy[['Name']])
df_copy[['Ticket']] = lc.fit_transform(df_copy[['Ticket']])

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [13]:
test_df = df_copy[train_len:]
X = df_copy[:train_len]
y = train.Survived

In [14]:
X[['Fare', 'Age']] = scaler.fit_transform(X[['Fare', 'Age']])
test_df[['Fare', 'Age']] = scaler.transform(test_df[['Fare', 'Age']])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1734: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value 

In [15]:
X.head()


,Pclass,Name,Age,SibSp,Parch,Ticket,Fare,Family_Size,isAlone,Sex_female,...,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Cabin_X,Embarked_C,Embarked_Q,Embarked_S,Embarked_X
0,1,17441,0.134273,2,0,49,-0.241442,3,0,0,...,0,0,0,0,0,0,0,0,1,0
1,3,3063,-0.446054,0,0,49,-0.439740,1,1,0,...,0,0,0,0,0,1,0,0,1,0
2,3,17798,-2.102432,1,2,14,0.393428,4,0,0,...,0,0,0,0,0,1,0,0,1,0
3,3,12742,-1.067320,0,0,0,-0.444197,1,1,0,...,0,0,0,0,0,1,0,0,1,0
4,3,2335,-0.734664,0,0,49,-0.520123,1,1,0,...,0,0,0,0,0,1,0,0,1,0


In [16]:
# def objective(trial, data=X, target=y):
    
# #     aucs = []
    
#     X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=0)
# #     skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

#     params = {
#             "random_state": 42,
#             "metric": "auc",
# #             "categorical_feature": cat_indices,
#             "verbosity": -1,
#             "n_estimators": 5000,
#             "learning_rate": trial.suggest_categorical('learning_rate', [0.001, 0.005, 0.01, 0.05, 0.1]),
#             'num_leaves': trial.suggest_int('num_leaves', 2, 1024),
#             'max_depth': trial.suggest_int('max_depth', -1, 32),
#             'reg_alpha': trial.suggest_float('reg_alpha', 1E-16, 25),
#             'reg_lambda': trial.suggest_float('reg_lambda', 1E-16, 25),
#             'colsample_bytree': trial.suggest_float('colsample_bytree', 1E-16, 1.0),
#             'subsample': trial.suggest_float('subsample ', 1E-16, 1.0),
#             'cat_smooth': trial.suggest_float('cat_smooth', 1.0, 50.0),
#             'device_type': 'gpu'
#     }
    
# #     for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
        
# #         print('Fold {}:'.format(fold))
        
# #         X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
# #         X_test, y_test = X.iloc[val_idx], y.iloc[val_idx]
    
#     lgb_model = LGBMClassifier(**params)
    
#     lgb_model.fit(X_train, y_train, eval_set=[(X_train, y_train),(X_test, y_test)],
#         eval_metric='auc', early_stopping_rounds=200, verbose=1000)
    
#     pred = lgb_model.predict_proba(X_test)[:,1]

#     auc = roc_auc_score(y_test, np.where(pred>0.5, 1, 0))
        
# #         aucs.append(auc)
#     return auc
# #     return np.average(aucs)

In [17]:
# %%time

# study = optuna.create_study(study_name="lgbm_classifier", direction="maximize")
# study.optimize(objective, n_trials=30)

In [18]:
# best_params = study.best_trial.params
# best_params['random_state'] = 42
# best_params['metric'] = 'auc'
# best_params['n_estimators'] = 5000
# best_params['verbosity'] = -1
# best_params

```{'learning_rate': 0.001,
 'num_leaves': 236,
 'max_depth': 10,
 'reg_alpha': 24.851800493638546,
 'reg_lambda': 20.10035504769574,
 'colsample_bytree': 0.8184743953551142,
 'subsample ': 0.4011776137391286,
 'cat_smooth': 34.824512621833776,
 'random_state': 42,
 'metric': 'auc',
 'n_estimators': 5000,
 'verbosity': -1}```

In [19]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=0)

# model = LGBMClassifier(**best_params)
# model.fit(X_train, y_train, eval_set=[(X_train, y_train),(X_test, y_test)],
#         eval_metric='auc', early_stopping_rounds=200, verbose=500)
# preds = model.predict(df_copy[train_len:])

In [20]:
# study.best_trial.params

In [21]:
# preds

In [22]:
# submission['Survived'] = preds
# submission = submission.set_index('PassengerId')
# submission.to_csv("april_tps_lgbm(3).csv")
# submission
